In [76]:
%gui qt
from dataclasses import dataclass

import ome_zarr
import zarr
import napari
from scipy.spatial import ConvexHull

import numpy as np





In [10]:
data = zarr.open("https://s3.embassy.ebi.ac.uk/idr/zarr/v0.1/6001247.zarr/", mode='r')

In [26]:
labels = data['labels'][0]

In [29]:
lbls = np.unique(labels)

In [27]:
arr = data[0]

In [68]:
@dataclass
class TriMesh:
    points: np.ndarray
    triangles: np.ndarray
    index: int = 0
    
    @property
    def colors(self):
        return np.linalg.norm(
            self.points - self.points.mean(axis=0)[None, :],
            axis=1
        )

    

In [92]:
meshes = []
for idx in lbls[1:6]:
    _, x, y, z = (labels[0] == idx).nonzero()
    points = np.vstack([x, y, z]).T
    hull = ConvexHull(points)
    triangles = hull.simplices
    meshes.append(TriMesh(points=points, triangles=triangles, index=idx))


In [75]:
viewer = napari.Viewer()
viewer.add_image(arr)

for mesh in meshes:
    viewer.add_surface((mesh.points, mesh.triangles, mesh.colors), colormap="Blues", opacity=1)

In [90]:
import meshio

In [99]:
mesh_ = meshes[0]
mesh = meshio.Mesh(
    points=mesh_.points,
    cells=[("triangle", mesh_.triangles),],
    point_data={"color": mesh_.colors})

In [83]:
import sys

In [84]:
sys.path.append("..")

In [109]:
meshio.__version__

'4.0.10'

In [113]:
cb = mesh.cells[0]

In [114]:
cb.data

array([[22734, 30359, 12514],
       [35905, 39779, 35567],
       [35905, 37173, 39779],
       ...,
       [30882, 24923, 33708],
       [30882, 38689, 38709],
       [30882, 30932, 38709]], dtype=int32)

In [85]:
import ply_zarr

ImportError: cannot import name 'cell_type_to_count' from 'meshio.ply._ply' (/home/guillaume/miniconda3/lib/python3.7/site-packages/meshio/ply/_ply.py)